In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import cv2

In [3]:
import numpy as np
import time
import glob, os, sys
import pickle
from subprocess import call

import numpy as np
from keras import backend as K
from keras.preprocessing import image
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers

from sklearn.model_selection import train_test_split

from keras.applications.resnet50 import ResNet50 as resnet
from keras.applications.resnet50 import preprocess_input as resnet_pp

In [1]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
def load_object(filename):
    with open(filename, 'rb') as fp:
        return pickle.load(fp)

In [2]:
# Build a model by adding preprocessing before the pretrained CNN
def get_feature_extraction_model():
    cnn_object, pp_function, img_size = _get_pretrained_model()
    model = keras.models.Sequential()
    cnn_model = cnn_object(weights='imagenet', include_top=False, pooling='max')
    model.add(keras.layers.Lambda(pp_function, name='preprocessing', input_shape=(img_size, img_size, 3)))
    model.add(cnn_model)
    return model

# Unpacking information from the models dictionary
def _get_pretrained_model():
    cnn_object = resnet
    pp_function = resnet_pp
    img_size = 224
    return cnn_object, pp_function, img_size

def get_features(files, model):
    # Load images based on the size of the Lambda layer 
    # provided as the first layer before the pretrained CNN
    x = np.array([image.img_to_array(image.load_img(file, target_size=(model.layers[0].input_shape[1], model.layers[0].input_shape[1]))) for file in files])
    return model.predict(x, verbose=1)